In [174]:
from utils import load_model

In [177]:
model_epochs = 1
img_height, img_width = 64, 64
model_path = f"ml_models/conv_nn_flower_classifier_{img_width}_{img_height}_{model_epochs}"
model = load_model(model_path)

In [178]:
model


In [164]:

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
# Helper libraries
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas
from scipy.ndimage import rotate
import os
import random

In [165]:
data = "ml_data/flower_photos/"

In [166]:
folders = os.listdir(data)

print(folders)

['roses', 'daisy', 'tulips', 'sunflowers']


In [181]:
train_labels, labels

(['roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',
  'roses',

In [182]:
# Import the images and resize them to a 128*128 size
# Also generate the corresponding labels

image_names = []
train_labels = []
train_images = []

size = 128,128

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue
      

Premature end of JPEG file


In [168]:
def random_crop(img, crop_size=(5, 5)):
    assert crop_size[0] <= img.shape[0] and crop_size[1] <= img.shape[1], "Crop size should be less than image size"
    img = img.copy()
    w, h = img.shape[:2]
    x, y = np.random.randint(h-crop_size[0]), np.random.randint(w-crop_size[1])
    img = img[y:y+crop_size[0], x:x+crop_size[1]]
    return img

In [169]:
def rotate_img(img, angle, bg_patch=(5,5)):
    assert len(img.shape) <= 3, "Incorrect image shape"
    rgb = len(img.shape) == 3
    if rgb:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1], :], axis=(0,1))
    else:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1]])
    img = rotate(img, angle, reshape=False)
    mask = [img <= 0, np.any(img <= 0, axis=-1)][rgb]
    img[mask] = bg_color
    return img

In [170]:
def translate(img, shift=10, direction='right', roll=True):
    assert direction in ['right', 'left', 'down', 'up'], 'Directions should be top|up|left|right'
    img = img.copy()
    if direction == 'right':
        right_slice = img[:, -shift:].copy()
        img[:, shift:] = img[:, :-shift]
        if roll:
            img[:,:shift] = np.fliplr(right_slice)
    if direction == 'left':
        left_slice = img[:, :shift].copy()
        img[:, :-shift] = img[:, shift:]
        if roll:
            img[:, -shift:] = left_slice
    if direction == 'down':
        down_slice = img[-shift:, :].copy()
        img[shift:, :] = img[:-shift,:]
        if roll:
            img[:shift, :] = down_slice
    if direction == 'up':
        upper_slice = img[:shift, :].copy()
        img[:-shift, :] = img[shift:, :]
        if roll:
            img[-shift:,:] = upper_slice
    return img


In [171]:
def gaussian_noise(img, mean=0, sigma=0.03):
    img = img.copy()
    noise = np.random.normal(mean, sigma, img.shape)
    mask_overflow_upper = img+noise >= 1.0
    mask_overflow_lower = img+noise < 0
    noise[mask_overflow_upper] = 1.0
    noise[mask_overflow_lower] = 0
    img += noise
    return img

In [173]:
for t, l in zip(train, labels):
    
    train = np.concatenate([np.expand_dims(translate(t, direction="up", shift=20), axis=0), train], axis=0)
    labels = np.append(labels, l)

    train = np.concatenate([np.expand_dims(translate(t, direction="down", shift=20), axis=0), train], axis=0)
    labels = np.append(labels, l)

    train = np.concatenate([np.expand_dims(translate(t, direction="left", shift=20), axis=0), train], axis=0)
    labels = np.append(labels, l)

    train = np.concatenate([np.expand_dims(translate(t, direction="right", shift=20), axis=0), train], axis=0)
    labels = np.append(labels, l)

    # train = np.concatenate([np.expand_dims(random_crop(t), axis=0), train], axis=0)
    # labels = np.append(labels, l)

    # train = np.concatenate([np.expand_dims(random_crop(t), axis=0), train], axis=0)
    # labels = np.append(labels, l)

    # train = np.concatenate([np.expand_dims(random_crop(t), axis=0), train], axis=0)
    # labels = np.append(labels, l)

    train = np.concatenate([np.expand_dims(rotate_img(t, 90), axis=0), train], axis=0)
    labels = np.append(labels, l)

    train = np.concatenate([np.expand_dims(rotate_img(t, 180), axis=0), train], axis=0)
    labels = np.append(labels, l)

    train = np.concatenate([np.expand_dims(rotate_img(t, 45), axis=0), train], axis=0)
    labels = np.append(labels, l)


KeyboardInterrupt: 

In [183]:
train = np.array(train_images)

train = train.astype('float32') / 255.0


In [184]:
label_dummies = pandas.get_dummies(train_labels)

labels =  label_dummies.values.argmax(1)

In [205]:
N = 2000
train_labels[N], labels[N]

('sunflowers', 2)

In [160]:
# Shuffle the labels and images randomly for better results

union_list = list(zip(train, labels))
random.shuffle(union_list)
train,labels = zip(*union_list)

# Convert the shuffled list to numpy array type

train = np.array(train)
labels = np.array(labels)

In [161]:
train.shape

(6739, 64, 64, 3)

In [162]:
# Develop a sequential model using tensorflow keras
img_height, img_width = 64, 64
num_classes = len(labels)

model = Sequential([
  # layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# Compute the model parameters

model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [163]:
model.fit(train,labels, epochs=15)

Epoch 1/15
211/211 [==============================] - 21s 97ms/step - loss: 1.1807 - accuracy: 0.6942
Epoch 2/15
211/211 [==============================] - 20s 95ms/step - loss: 0.9278 - accuracy: 0.7338
Epoch 3/15
211/211 [==============================] - 21s 98ms/step - loss: 0.9244 - accuracy: 0.7338
Epoch 4/15
211/211 [==============================] - 24s 112ms/step - loss: 0.9213 - accuracy: 0.7338
Epoch 5/15
211/211 [==============================] - 21s 99ms/step - loss: 0.9213 - accuracy: 0.7338
Epoch 6/15
211/211 [==============================] - 20s 93ms/step - loss: 0.9199 - accuracy: 0.7338
Epoch 7/15
211/211 [==============================] - 20s 95ms/step - loss: 0.9419 - accuracy: 0.7338
Epoch 8/15
211/211 [==============================] - 20s 95ms/step - loss: 0.9307 - accuracy: 0.7338
Epoch 9/15
211/211 [==============================] - 20s 93ms/step - loss: 0.9282 - accuracy: 0.7338
Epoch 10/15
211/211 [==============================] - 20s 96ms/step - loss: 0.92